# PassGAN

This Jupyter notebook is mainly for debugging, has same functionality to "sample.py".

## Dependencies

In [1]:
# Libraries
import os
import time
import pickle
import argparse
import tensorflow as tf
import numpy as np

# Files
import tflib as lib
import tflib.ops.linear
import tflib.ops.conv1d
import utils
import models

In [2]:
# TensorFlow virsion
print(tf.__version__)

1.13.1


## Parameters

To simplify migration process between Jupyter Notebook and .py format, we created virtual ArgumentParser class.

In [3]:
# Class to virtualize ArgumentParser
class VirtualArgparse:
    
    # Name of directory where pickle dictyonary is stored
    input_dir = "pretrained"
    
    # TensorFlow checkpoint
    checkpoint = "pretrained/checkpoints/checkpoint_200000.ckpt"
    
    # Name of file to output generated passwords
    output = "generated_pass.txt"
    
    # Batch size
    batch_size = 1024
    
    # Number of password to generate
    num_samples = 1000000
    
    # Max length of password to generate (Training-Dependent)
    seq_length = 10
    
    # Number of nodes on hidden layers (Training-Dependent)
    layer_dim = 128

In [4]:
# Virtualize ArgumentParser instance
args = VirtualArgparse()

## Importing Dictionary

In [5]:
# Dictionary
with open(os.path.join(args.input_dir, 'charmap.pickle'), 'rb') as f:
    charmap = pickle.load(f, encoding='latin1')

# Reverse-Dictionary
with open(os.path.join(args.input_dir, 'charmap_inv.pickle'), 'rb') as f:
    inv_charmap = pickle.load(f, encoding='latin1')

## Creating Generator

In [6]:
fake_inputs = models.Generator(args.batch_size, args.seq_length, args.layer_dim, len(charmap))

Instructions for updating:
Colocations handled automatically by placer.


## Starting TensorFlow Session

In [7]:
with tf.Session() as session:

    def generate_samples():
        samples = session.run(fake_inputs)
        samples = np.argmax(samples, axis=2)
        decoded_samples = []
        for i in range(len(samples)):
            decoded = []
            for j in range(len(samples[i])):
                decoded.append(inv_charmap[samples[i][j]])
            decoded_samples.append(tuple(decoded))
        return decoded_samples

    def save(samples):
        with open(args.output, 'a') as f:
                for s in samples:
                    s = "".join(s).replace('`', '')
                    f.write(s + "\n")

    saver = tf.train.Saver()
    saver.restore(session, args.checkpoint)

    samples = []
    then = time.time()
    start = time.time()
    for i in range(int(args.num_samples / args.batch_size)):
        
        samples.extend(generate_samples())

        # append to output file every 1000 batches
        if i % 1000 == 0 and i > 0: 
            
            save(samples)
            samples = [] # flush

            print('wrote {} samples to {} in {:.2f} seconds. {} total.'.format(1000 * args.batch_size, args.output, time.time() - then, i * args.batch_size))
            then = time.time()
    
    save(samples)
print('\nFinished in {:.2f} seconds'.format(time.time() - start))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from pretrained/checkpoints/checkpoint_200000.ckpt

Finished in 24.89 seconds
